# Métricas

Para evaluar los dos modelos creados previamente primero nos centraremos en crear un conjunto de prueba siguiendo los siguientes pasos:

1. Obtener un conjunto de oraciones variadas (C)
    - Oraciones simples
    - Oraciones complejas 
    - Oraciones con ruido

2. TP - que recupere la query tal cual o lo más parecido
    - La query (c in C) si se encuentra en corpus
    - La query se encuentra parcialmente
       (tomamos oraciones del corpus y le metemos ruido)
       (tendríamos que revisar que el corpus si tenga la query parcial
       y luego revisar los resultados, si la busqueda da resultados
       pero en corpus no hay entonces es un falso positivo)


3. TN - oraciones donde ninguna palabra está dentro del corpus
    - Si regresa resultados es un falso negativo 

<p align="center">
  <img src="img/metricas.jpg" width="70%" alt="Metricas"/>
</p>

In [23]:
from os import listdir
from os.path import isfile, join
import json
from sklearn.model_selection import train_test_split #particiones
import string
import random
import pickle

Cargamos el corpus original para seleccionar algunas oraciones

In [31]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
        with open(path+"/"+file, 'rb') as f:
            text.append(json.loads(f.read().decode('utf-8', 'replace')))
    return text

# Guardamos los subtitulos de todos los canales
videos_original = get_txt("../corpus/data")#Nota: no print de todo
# Obtenemos el conjunto de test
_, test = train_test_split(videos_original, test_size=0.007) #5%

In [32]:
def get_subtitles():
    '''
    Regresa los subtitulos del corpus
    '''
    test_subtitles = []
    for chanel in test:
        for video in chanel:
            if 'subtitles' in video:
                for s in video['subtitles']:
                    test_subtitles.append(s['text'])
    return test_subtitles

Del 2.5% de oraciones tomadas del corpus tomamos la mitad para meterles ruido

In [33]:
test_TP, test_TPR = train_test_split(get_subtitles(), test_size=0.5)
print(test_TPR[:10], len(test_TPR), len(test_TP))

['descubrir todo aquello que te gusta', 'detrás de escena de cómo están hechos', 'bueno más o menos', 'estas películas es historia de mi', 'mismo idioma', 'competencia por decir tenemos audiciones', 'portafolio muerto', '[Música]', 'como es tener acceso exclusivo a grandes', 'donde mana los sueños que van'] 1814 1814


In [34]:
def insert(s, num):
    """
    Agrega n cadenas aleatorias a una lista

    Args:
        s (list): lista de cadenas
        num (int): número de cadenas aleatorias a insertar
    """
    for _ in range(num):
        noise = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(6))
        ix = random.randint(0,len(s)-1)
        s.insert(ix, noise)

def make_noise():
    """
    A cada oración del conjunto le agrega n cadenas aleatorias
    """
    for i, sentence in enumerate(test_TPR):
        s = sentence.split(" ")
        insert(s, 2)
        test_TPR[i] = ' '.join(s)

make_noise()
print(test_TPR[:10], len(test_TPR))

['descubrir Y9zK4T todo aquello que te AXtyjr gusta', 'detrás XutpTq de escena de cómo están tK0PIg hechos', 'bueno más mCTdV4 qqTfE4 o menos', 'estas películas Fs38d8 es historia 1fHzOx de mi', 'qXAXH6 vyOh1z mismo idioma', 'competencia pb5X8F por nsB0iZ decir tenemos audiciones', '0MKtio fHPsNT portafolio muerto', 'Fo1GNF JBbZoF [Música]', 'como es tener acceso iibbdR zProHZ exclusivo a grandes', 'donde YgHLSP mana los pA8QME sueños que van'] 1814


Generamos 1.25% de oraciones con palabras que no están en el corpus de tal manera que el conjunto de prueba queda compuesto por:
- 1.25% oraciones que se encuentran tal cual en el corpus
- 1.25% oraciones que se encuentran  en el corpus pero con ruido
- 0.625% oraciones que no comparten palabras con el corpus

In [35]:
def make_sentence():
    """
    Crea un conjunto de cadenas aleatorias
    """
    sentence = []
    lg = [3,5,10,20]
    for _ in range(len(test_TP)//2):
        s = [""]
        insert(s, lg[random.randint(0,len(lg)-1)])
        sentence.append(' '.join(s))
    return sentence
    
test_TN = make_sentence()
print(test_TN[:10], len(test_TN))

['bnDz4Z vULMUu qnYG0r jprEgi pvuvJq JX3bQF ssGsSh Bizp0T 6ilRIt 8eWyqO GVfXsW nvboUr EJFOK3 VRbl6q npKIoU BZj9FN RFf9g7 Kx7sv1 rTcMQS eObTui ', 'THwuqq 7q9yXL RGTevs ', 'rBlL2C SfPeKx Mhr1WT ziJTXi f0QJlT g7ORm3 2yKxwl KgeAua ONAECf HUbSib ', 'aClHOo Dpym7P hZ22hT 2RmlyO PU06cP Zuax8u 1Elo9d ac24UI cIpuzN EAztti ', 'ePSvZY pRNBfZ 4UsmiA 1pEjgf 7DHVAJ BhvZYH SJKwZ7 lmOzl9 oEvNr3 pTnRau ', 'zmn7Tm YNN7QL pdiUJa lvJl64 lj6z6l XhocVQ 6i118x xV9vrK 6p0Mko rmCGFP KcCKNe VlzynS Y8Mz6E i2XIDa 0EUo0N 3Cfq0C iB7jjy IZHVQK V9oNhV yPY1gR ', 'bLJQeL lPTMG3 QXVI2T BZR1EG T34yGc OF2bwm 3Sd19V jQj3vb zitnPD 5qJyTO ', '66m5pA TW7yQm 1cYFUf DgNXJM P4a4DJ 2LmfNq TV3hu5 jkOIOv 8K0oOH 3GBJco ', '1vQvmz NDi2hb JahNiu BedsZJ 7GChSI 7hYMmu JIS8XH KgT0DJ E8VZCH WFv5qC euOYyT 0oi7Up jYLmcV OxKoNt zD6Gae oxGau6 0L7gZC BgqUzw O2gyev IVlPWd ', 'Lsx61H qTH8JD 7TbByM 80M8ba D90Nih '] 907


Guardamos el conjunto de prueba

In [36]:
test_all = test_TP + test_TPR + test_TN
pickle.dump(test_all, open("../pkl/test.pkl", "wb"))

In [40]:
with open("../pkl/test.pkl", "rb") as f:
    test_all = pickle.load(f)

test_all[3000:3050]

['blJsis 8eaxn6 asco',
 'l0nrhs LEt41H [Aplausos]',
 'NRKCrq vida como GZRafD detective aquí es donde vivo a',
 'zpty5d LCNCRi más o menos',
 'z1IRhf voy a comerte ZSquts vivo',
 'vamos a celebrar con una gran fiesta qW7Kal RHrYMU un',
 'estos son más CK4oNF que objetos V90QM1 y vestuarios',
 '15 mil 5cFBMT dólares Y8oMHy por favor sin el lado',
 'mundo la apertura del concierto estará O9xVT4 fmqnb9 a',
 'deberíamos 1GdCyN tratar de jFROLG averiguar dónde',
 'Hz9q70 LxbqQR [Aplausos]',
 'aparte están dSdqqk aEB6z2 las personas que desean',
 'VAzBvh n96dw0 [Música]',
 'FQzQ69 uh3qPs [Música]',
 'caminan a 8goHyq fGBiAu madrid aplausos',
 'OeigtJ maravilloso su reto será QGywUK el lograr con',
 'X8Q3qV también tienen momentos s7c3OF turbios cuánto',
 'fiE4Z6 6zxctO estuviste cerca',
 'ella buscar a 5eKhqJ mi hermano y 6OeY4D tú vengas',
 'su rShTIy i4yoCk cabeza verdad la mía es la cultura yo',
 'ard0UB tienen fJX1b2 superpoderes somos superhéroes',
 'el océano te IPtBNu eligió por soJ17

In [38]:
len(test_all)

4535

Ahora veremos que tan bien recupera las oraciones de prueba

In [23]:
def count_words(s1, s2):
    """
    Regresa el número de palabras que conparten 2 cadenas

    Args:
        s1 (lst): lista de palabras de la cadena 1
        s2 (lst): lista de palabras de la cadena 2
    """
    val = 0
    for i in s2: 
        if i in s1: val += 1
    return val

In [24]:
count_words("a un lado oigan alejen a los demás viene".split(), 'y fue porque íbamos a saltar de los'.split())

2

In [18]:
'y' in 'fue porque íbamos a saltar de los'

False

In [29]:
#Para el caso de TFIDF cuando no encuentra coincidencias
#(score = 0 para todas las cadenas)regresa lo siguiente:
dummy_sentence = ['tu papá tuvo un accidente', 'te recuerdo que está', 
                  'aquí', 'estaba como rabioso con', 
                  'especial con lucas martín']

def eval_sentence(query, resp):
    """
    Evalúa los resultados obtenidos dada una query

    Args:
        query (str): query de prueba generada previamente
        resp (list): lista con las primeras 5 respuestas obtenidas 
            por algún mecanismo de RI
    Return:
        (int): 0 cuando no encontró ninguna coincidencia
               1 si encontró la query exacta y es el primer resultado
               (0 - 1) si:
                - Encuentra la query pero no es el primer resultado
                - No se encuentra la query pero algunas palabras de la misma sí 
    """
    val = 0 
    total = 0

    if resp != dummy_sentence: 
        if resp[0] == query:
            val = 1
        else:
            count = 0
            for r in resp:
                total += len(r)
                count += count_words(query.split(), r.split())
            val = count / total 
    return val

Probamos los 3 casos

In [27]:
eval_sentence("avergonzado lo suficiente no apenas",
             ['avergonzado lo suficiente no apenas', 
             'pesados o polar no lo suficiente como', 
             'no vende suficiente leña lo siento pero', 
             'Como pueden ver, no me cuesta trabajo \nestirar la masa porque descanso lo suficiente.', 
             'aún así no es suficiente porque lo'])

1

In [31]:
eval_sentence("a un lado oigan alejen a los demás viene",   
             ['Vámonos a casa.', 
             '¿Cómo ibas a saberlo?', 
             '¡Hola a todos!', 
             'Son fisuras\na un infierno desconocido.', 
             'y fue porque íbamos a saltar de los'])

0.056910569105691054

In [32]:
eval_sentence("8eIqyc xwWsUF 3eHwS8 qE5XFM Tknsyj",
             ['tu papá tuvo un accidente', 
             'te recuerdo que está', 
             'aquí', 
             'estaba como rabioso con', 
             'especial con lucas martín'])

0

In [1]:
1814*2

3628

In [ ]:
def eval_all(resp):
    """
        Si está en el conjunto 1 (original) debería de responder con 1
            Si 1 TP++ else si 0 FN++
        Si esta en el conjunto 2 (con ruido) debería responder con un número decimal
            Si mayor a .5 TP++ else si menor a 5 FN++
        Si está en el conjunto 3 (que no están) deberia responder con 0
            Si 0 TN++ else si 1 FP++
    """
    TP, FN, FP, TN = 0, 0, 0, 0
    #Recuperamos las particiones
    """for query, resp in tqdm(zip(test, resp)):
        print()"""
        #Hacer las particiones de los conjuntos
        

In [41]:
def get_metrics(TP, FN, FP, TN):
    """
    Realiza el cálculo de:
    Sensitive, specificity, precision, 
    negative predictive value y accuracy 

    Args:
        TP (int): True Positive
        FN (int): False Negative
        FP (int): False Positive
        TN (int): True Negative
    """
    return {"Sensitive": TP / (TP + FN),
            "Specificity": TN / (TN + FP),
            "Precision": TP / (TP + FP),
            "Negative predictive value": TN / (TN + FN),
            "Accuracy": (TP + TN) / (TP + TN + FP + FN)}